# Unitary Decomposition Based on JanusQ-CT

**Author:** Congliang Lang \& Siwei Tan  

**Date:** 8/4/2024

Based on "[QuCT: A Framework for Analyzing Quantum Circuit by Extracting Contextual and Topological Features (MICRO 2023][1]"

[1]: https://scholar.google.com/scholar_url?url=https://dl.acm.org/doi/abs/10.1145/3613424.3614274%3Fcasa_token%3DffjIB1hQ4ZwAAAAA:8MajDLrDOC74WoeMf7r7AoQ-koxCa4E1TNqQg3GSDz03xUX6XdE3toNTM-YdM_e4rKEusMceJ6BGJg&hl=zh-CN&sa=T&oi=gsb&ct=res&cd=0&d=11146218754516883150&ei=42YSZpPlFL6s6rQPtt6x6Ac&scisig=AFWwaeYaiu2hyx8HUJ_7Buf9Mwom

The expressiveness of gate vectors enables JanusQ-CT to reconstruct sub-circuits via gate vectors, which is a necessary ability for tasks involving circuit generation, such as the unitary decomposition. Unitary decomposition is a task that takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.

In [1]:
import sys
sys.path.append('..')
import os
os.chdir("..")
import logging
logging.basicConfig(level=logging.WARN)
import ray
ray.init(log_to_driver=False)
from qiskit.quantum_info import random_unitary
from janusq.data_objects.circuit import qiskit_to_circuit
from janusq.data_objects.backend import  LinearBackend
from janusq.analysis.vectorization import RandomwalkModel
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.unitary_decompostion import decompose
import time

2024-04-19 02:16:27,436	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67067904 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-04-19 02:16:28,586	INFO worker.py:1724 -- Started a local Ray instance.


## Construct U2V model

The U2V model serves as the bridge between unitaries and gate vectors, where the sub-circuits reconstructed from these candidate vectors will replace the search space of QFAST. To build such a model, we obtain a U2V dataset composed of <$unitary,~\{vectors\}$> pairs, derived from a set of random circuits generated with the same scheme mentioned in Section~\ref{sec:equ_prediction}.

In [2]:
n_qubits = 5
backend = LinearBackend(n_qubits, 1, basis_two_gates = ['crz'])

In [4]:

from janusq.analysis.unitary_decompostion import U2VModel

n_step = 2

dataset = random_circuits(backend, n_circuits=50, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

vec_model = RandomwalkModel(
    n_step, 4 ** n_step, backend, directions=('parallel', 'next'))
vec_model.train(dataset, multi_process=True,
                        remove_redundancy=False)

u2v_model = U2VModel(vec_model)
data = u2v_model.construct_data(dataset, multi_process=False)
u2v_model.train(data, n_qubits)


100%|██████████| 6/6 [00:01<00:00,  5.17it/s]


len(Us) =  907 len(gate_vecs) =  907
Start construct U2VMdoel
Finish construct U2VMdoel, costing 1.031064748764038s


## Decompose a unitary

In [5]:
# generate a random unitary
unitary = random_unitary(2**n_qubits).data

# apply decomposition
start_time = time.time()
quct_circuit = decompose(unitary, allowed_dist = 0.2, backend = backend, u2v_model = u2v_model)
quct_time = time.time() - start_time


print(quct_time)

str(quct_circuit)

100%|██████████| 1/1 [01:02<00:00, 62.43s/it]

solution_dist 0.7670839842239192


323.8224334716797


'       ┌──────────────────────────┐  ░               ░ »\nq_0: ──┤ U(0.3714,0.07482,2.3743) ├──░───────────────░─»\n     ┌─┴──────────────────────────┴┐ ░               ░ »\nq_1: ┤ U(-1.2813,0.098748,0.11357) ├─░───────────────░─»\n     └┬────────────────────────────┤ ░               ░ »\nq_2: ─┤ U(-1.8659,-0.68533,1.3887) ├─░───────■───────░─»\n      └─┬────────────────────────┬─┘ ░ ┌─────┴─────┐ ░ »\nq_3: ───┤ U(2.519,0.311,0.70299) ├───░─┤ Rz(3.632) ├─░─»\n        └────────────────────────┘   ░ └───────────┘ ░ »\nq_4: ────────────────────────────────░───────────────░─»\n                                     ░               ░ »\n«                                    ░              ░ »\n«q_0: ───────────────────────────────░──────────────░─»\n«                                    ░              ░ »\n«q_1: ───────────────────────────────░──────■───────░─»\n«     ┌────────────────────────────┐ ░      │       ░ »\n«q_2: ┤ U(-2.1458,1.7739,-0.68533) ├─░──────┼───────░─»\n«     ├────────────

In [6]:
# compare it with the qsd method
from qiskit.synthesis.unitary.qsd import qs_decomposition

start_time =time.time()
qc = qs_decomposition(unitary)

qsd_circuit = qiskit_to_circuit(qc)
qsd_time = time.time() - start_time

print(qsd_time)

str(qsd_circuit)

ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:unkown gate
ERROR:root:

0.5025274753570557


'                          ░       ░                    ░       ░ »\nq_0: ─────────────────────░───■───░────────────────────░───────░─»\n                          ░   │   ░                    ░       ░ »\nq_1: ─────────────────────░───┼───░────────────────────░───■───░─»\n     ┌─────────────────┐  ░ ┌─┴─┐ ░ ┌────────────────┐ ░ ┌─┴─┐ ░ »\nq_2: ┤ U(0,0,-0.13056) ├──░─┤ X ├─░─┤ U(0,0,-1.7746) ├─░─┤ X ├─░─»\n     ├─────────────────┴┐ ░ └───┘ ░ └────────────────┘ ░ └───┘ ░ »\nq_3: ┤ U(0,0,-0.007322) ├─░───────░────────────────────░───────░─»\n     ├─────────────────┬┘ ░       ░                    ░       ░ »\nq_4: ┤ U(0,0,0.077985) ├──░───────░────────────────────░───────░─»\n     └─────────────────┘  ░       ░                    ░       ░ »\n«                        ░       ░                     ░       ░ »\n«q_0: ───────────────────░───■───░─────────────────────░───────░─»\n«                        ░   │   ░                     ░       ░ »\n«q_1: ───────────────────░───┼───░─────────────

In [7]:
synthesis_method_result = [qsd_circuit,  quct_circuit]
synthesis_method_time = [qsd_time,  quct_time]
for res, tim in zip(synthesis_method_result, synthesis_method_time):
    print(f"#gate: {res.n_gates}, #two_qubit_gate: {res.num_two_qubit_gate}, depth: {res.depth}, time: {tim} \n")

#gate: 897, #two_qubit_gate: 379, depth: 815, time: 0.5025274753570557 

#gate: 108, #two_qubit_gate: 34, depth: 52, time: 323.8224334716797 

